In [6]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

behavioral_responses = Reform.from_dict({
  "gov.simulation.labor_supply_responses.elasticities.income": {
    "2024-01-01.2100-12-31": -0.05
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
    "2024-01-01.2100-12-31": 0.31
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
    "2024-01-01.2100-12-31": 0.28
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
    "2024-01-01.2100-12-31": 0.27
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
    "2024-01-01.2100-12-31": 0.27
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
    "2024-01-01.2100-12-31": 0.25
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
    "2024-01-01.2100-12-31": 0.25
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
    "2024-01-01.2100-12-31": 0.22
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
    "2024-01-01.2100-12-31": 0.22
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
    "2024-01-01.2100-12-31": 0.22
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
    "2024-01-01.2100-12-31": 0.27
  },
  "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
    "2024-01-01.2100-12-31": 0.22
  }
}, country_id="us")

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

baseline_behavioral = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")


In [7]:
repeal_it_ded_reform = Reform.from_dict(
    {
        "gov.irs.deductions.deductions_if_itemizing": {
            "2024-01-01.2100-12-31": ["wagering_losses_deduction", "qualified_business_income_deduction"]
        },
        "gov.irs.deductions.itemized_deductions": {
            "2024-01-01.2100-12-31": ["heat_pump_expenditures"]
        }
    }
)

repeal_medical_exp_reform = Reform.from_dict(
    {
        "gov.irs.deductions.itemized.medical.floor": {
            "2024-01-01.2100-12-31": 1_000_000_000
        }
    }
)

repeal_chart_ded_reform = Reform.from_dict(
    {
        "gov.irs.deductions.itemized.charity.ceiling.all": {
            "2024-01-01.2100-12-31": 0
        }
    }
)

repeal_interest_ded_reform = Reform.from_dict(
    {
        "gov.irs.deductions.deductions_if_itemizing": {
            "2024-01-01.2100-12-31": ["medical_expense_deduction", "charitable_deduction", "salt_deduction", "casualty_loss_deduction", "qualified_business_income_deduction", "wagering_losses_deduction"]
        },
        "gov.irs.deductions.itemized_deductions": {
            "2024-01-01.2100-12-31": ["medical_expense_deduction", "charitable_deduction", "salt_deduction", "casualty_loss_deduction"]
        }
    }
)

repeal_salt_ded_reform = Reform.from_dict(
    {
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        }
    }
)

In [8]:
# Dictionary of all the reforms
reforms = {
    "repeal_it_ded": repeal_it_ded_reform,
    "repeal_medical_exp": repeal_medical_exp_reform,
    "repeal_chart_ded": repeal_chart_ded_reform,
    "repeal_interest_ded": repeal_interest_ded_reform,
    "repeal_salt_ded": repeal_salt_ded_reform,
}


In [9]:
baseline_income = baseline.calculate("household_net_income", period=2024)


In [10]:
for name, reform in reforms.items():
    sim = Microsimulation(reform=reform,
                          dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    reformed_income = sim.calculate("household_net_income", period=2024)
    impact = (reformed_income - baseline_income).sum()
    print(f"Reform: {name} -> Impact on income tax: {impact:,.2f}")

Reform: repeal_it_ded -> Impact on income tax: -87,298,748,961.58
Reform: repeal_medical_exp -> Impact on income tax: -4,548,085,250.17
Reform: repeal_chart_ded -> Impact on income tax: -67,633,722,372.78
Reform: repeal_interest_ded -> Impact on income tax: -28,691,421,174.73
Reform: repeal_salt_ded -> Impact on income tax: -10,118,681,939.88


In [11]:
# First, re-calculate the baseline income tax in case you haven't already:
baseline_income = baseline.calculate("income_tax", period=2024)

# Loop over each reform, combine it with behavioral responses, and compute the impact
for name, reform in reforms.items():
    # Merge the specific reform with the behavioral responses.
    # Here we assume that each 'Reform' object exposes a to_dict() method to get its underlying dict.
    combined_reform_dict = {**reform.to_dict(), **behavioral_responses.to_dict()}
    combined_reform = Reform.from_dict(combined_reform_dict, country_id="us")
    
    # Initialize a new microsimulation with the combined reform
    sim = Microsimulation(
        reform=combined_reform,
        dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
    )
    
    # Calculate the income tax for the combined reform scenario
    reformed_income = sim.calculate("income_tax", period=2024)
    
    # Compute the impact. Here we use the same arithmetic as before.
    impact = (reformed_income - baseline_income).sum()
    print(f"Combined Reform: {name} with behavioral responses -> Impact on income tax: {impact:,.2f}")

AttributeError: type object 'reform' has no attribute 'to_dict'

In [7]:
# Calculate the baseline income tax (if not already done)
baseline_income = baseline.calculate("income_tax", period=2024)

# Define the original individual reform dictionaries
reforms_data = {
    "repeal_it_ded": {
        "gov.irs.deductions.deductions_if_itemizing": {
            "2024-01-01.2100-12-31": ["wagering_losses_deduction", "qualified_business_income_deduction"]
        },
        "gov.irs.deductions.itemized_deductions": {
            "2024-01-01.2100-12-31": ["heat_pump_expenditures"]
        }
    },
    "repeal_medical_exp": {
        "gov.irs.deductions.itemized.medical.floor": {
            "2024-01-01.2100-12-31": 1_000_000_000
        }
    },
    "repeal_chart_ded": {
        "gov.irs.deductions.itemized.charity.ceiling.all": {
            "2024-01-01.2100-12-31": 0
        }
    },
    "repeal_interest_ded": {
        "gov.irs.deductions.deductions_if_itemizing": {
            "2024-01-01.2100-12-31": [
                "medical_expense_deduction", "charitable_deduction", "salt_deduction",
                "casualty_loss_deduction", "qualified_business_income_deduction", "wagering_losses_deduction"
            ]
        },
        "gov.irs.deductions.itemized_deductions": {
            "2024-01-01.2100-12-31": [
                "medical_expense_deduction", "charitable_deduction", "salt_deduction", "casualty_loss_deduction"
            ]
        }
    },
    "repeal_salt_ded": {
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        }
    }
}

# Define behavioral responses as a dictionary
behavioral_responses_data = {
    "gov.simulation.labor_supply_responses.elasticities.income": {
        "2024-01-01.2100-12-31": -0.05
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
        "2024-01-01.2100-12-31": 0.31
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
        "2024-01-01.2100-12-31": 0.28
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
        "2024-01-01.2100-12-31": 0.27
    }
}

# Loop over each individual reform, merge it with behavioral responses, and compare to baseline
for name, reform_dict in reforms_data.items():
    # Combine the individual reform with behavioral responses.
    combined_dict = {**reform_dict, **behavioral_responses_data}
    
    # Create a combined Reform from the merged dictionary
    combined_reform = Reform.from_dict(combined_dict, country_id="us")
    
    # Initialize the microsimulation using the combined reform
    sim = Microsimulation(
        reform=combined_reform,
        dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
    )
    
    # Calculate the income tax for the combined scenario
    reformed_income_tax = sim.calculate("income_tax", period=2024)
    
    # Compute the impact relative to the baseline (using the same arithmetic as before)
    impact = (reformed_income_tax - baseline_income).sum()
    print(f"Combined Reform: {name} with behavioral responses -> Impact on income tax: {impact:,.2f}")

Combined Reform: repeal_it_ded with behavioral responses -> Impact on income tax: 80,194,493,082.39
Combined Reform: repeal_medical_exp with behavioral responses -> Impact on income tax: 3,313,528,155.87
Combined Reform: repeal_chart_ded with behavioral responses -> Impact on income tax: 62,512,730,450.91
Combined Reform: repeal_interest_ded with behavioral responses -> Impact on income tax: 25,320,345,309.85
Combined Reform: repeal_salt_ded with behavioral responses -> Impact on income tax: 8,884,419,415.79
